## DMRG

For the half filling case $n_i=2$, the dimension is $C_4^2=6$ for the local Hilbert space. A basis of the states on a single site can be chosen as: 
$$
\begin{aligned}
& |a\rangle=|12\rangle, \quad|b\rangle=|13\rangle, \quad|c\rangle=|14\rangle, \\
& |d\rangle=|23\rangle, \quad|e\rangle=|24\rangle, \quad|f\rangle=|34\rangle. \\
&
\end{aligned}
$$
where $|12\rangle=c_1^\dagger c_2^\dagger|0\rangle$. 

The 15 traceless $SU(4)$ spin generators can be written as 
$$
S_{\alpha\beta} = c_\alpha^\dagger c_\beta - \frac{1}{2} \delta_{\alpha\beta}
$$
where $\alpha,\beta=1,2,3,4$. Let us carry out the matrix representation of these generators under the basis above. 

The diagonal terms are $S_{\alpha\alpha} = c_\alpha^\dagger c_\alpha - \frac{1}{2}$. One should calculate a large amount of expectations like $\langle a | c_1^\dagger c_1 | a\rangle$. For every $\alpha$ there are $6\times 6=36$ entries await. Thus it is better to accelerate it with a code. 

In [13]:
import numpy as np
from copy import deepcopy

def oprs_on_ket(oprs_original, ket):
    """
    Inputs: 
        1. opr, list of strings, the operators in the middle
        2. ket, list of strings, the ket

    Outputs:
        1. coe, the coefficient, 1 or -1 or 0
        2. ket, list of strings or zero(integer), the result of application of operators
    """

    oprs = deepcopy(oprs_original)
    assert len(oprs) != 0 #there must be at least one operator inside list oprs
    coe = 1

    while len(oprs) != 0:
        opr = oprs[-1]

        if opr.endswith('d'): #creation operator
            opr = opr[:-1] #cut the 'd' in the end
            if any(opr in _ for _ in ket): #if opr is already in ket
                ket = 0
                return 0, ket #return (0,0)
            else: #opr is not in ket
                ket.insert(0,opr)
        else: #annihilation operator
            if any(opr in _ for _ in ket): 
                index = ket.index(opr) #index is the number of particles 'before' opr
                coe *= (-1)**index
                ket.remove(opr)
            else:
                ket = 0
                return 0, ket #return (0,0)
        oprs = oprs[:-1] #cut the operator in oprs after application
    return coe, ket

def get_ket_from_6states(n):
    if n == 1:
        return ['1','2']
    elif n == 2:
        return ['1','3']
    elif n == 3:
        return ['1','4']
    elif n == 4:
        return ['2','3']
    elif n == 5:
        return ['2','4']
    elif n == 6:
        return ['3','4']
    else:
        raise("Out of 6 states. ")

def S_representation_matrix(alpha, beta):
    if type(alpha) != int or type(beta) != int or alpha>4 or alpha<1 or beta>4 or beta<1:
        raise("Check your alpha and beta. They must be 1,2,3,4. ")
    S_mat = np.zeros((6,6))
    for left in range(1,7):
        bra = get_ket_from_6states(left)
        oprs = [str(alpha)+'d', str(beta)]
        oprs.insert(0, bra[0])
        oprs.insert(0, bra[1]) #put the annihilation operators in the front of oprs
        for right in range(1,7):
            ket = get_ket_from_6states(right)
            coe, ket = oprs_on_ket(oprs, ket)
            if ket == []:
                S_mat[left-1, right-1] = coe
            elif ket == 0:
                S_mat[left-1, right-1] = 0
            else:
                raise('something wrong')
    if alpha==beta:
        S_mat -= (1/2)*np.diag([1,1,1,1,1,1])
    return S_mat

In [14]:
operators = ['3','2','1d','1']
state = ['2','3']

oprs_on_ket(operators, state)

S_representation_matrix(1,1)

array([[ 0.5,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -0.5,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. , -0.5,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. , -0.5]])

In [16]:
for a in range(1,5):
    for b in range(1,5):
        print("S",a,b,"matrix is")
        print(S_representation_matrix(a,b))

S 1 1 matrix is
[[ 0.5  0.   0.   0.   0.   0. ]
 [ 0.   0.5  0.   0.   0.   0. ]
 [ 0.   0.   0.5  0.   0.   0. ]
 [ 0.   0.   0.  -0.5  0.   0. ]
 [ 0.   0.   0.   0.  -0.5  0. ]
 [ 0.   0.   0.   0.   0.  -0.5]]
S 1 2 matrix is
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
S 1 3 matrix is
[[ 0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
S 1 4 matrix is
[[ 0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
S 2 1 matrix is
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
S 2 2 matrix is
[[ 0.5  0.   0.   0.   0.   0. ]
 [ 0.  -0.5  0.   0.   0.   0. ]
 [ 0.   0.  -0.5  0.   0.   0. ]
 [ 0.   0.   0.   0.5  0. 